In [83]:
def imports():
    from sklearn.feature_extraction.text import CountVectorizer
    from gensim.corpora import Dictionary
    from gensim.models.ldamodel import LdaModel
    from gensim.models import CoherenceModel
    from nltk.corpus import stopwords
    from nltk.tokenize import RegexpTokenizer
    from datetime import datetime
    
    import pandas as pd
    import re
    import math
    import numpy as np # linear algebra
    import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
    import os
    
    import nltk
    nltk.download('stopwords')
    
    from nltk.stem.snowball import SnowballStemmer

    import contractions 
    nltk.download('wordnet')
    #CONFIG
    pd.set_option('display.max_colwidth', None)
    input_path = '/home/giangvdq/data/NIPS Papers'
    os.chdir(input_path)
    
imports()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/giangvdq/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [96]:
#initialize

papers = pd.read_csv('papers.csv')
authors2 = pd.read_csv('authors.csv')
paper_authors = pd.read_csv('paper_authors.csv')

In [25]:
def preprocess(df, tweet_col):
    df_copy = df.copy()
    
    # drop rows with empty values
    df_copy.dropna(inplace=True)
    
    # format the date
    df_copy[date_col] = df_copy[date_col].apply(lambda row: datetime.strptime(row, '%m-%d-%Y %H:%M:%S'))
    
    # filter rows older than a given date
    df_copy = df_copy[df_copy[date_col] >=start_datetime]
    
    # lower the tweets
    df_copy['preprocessed_' + tweet_col] = df_copy[tweet_col].str.lower()
    
    # filter out stop words and URLs
    en_stop_words = set(stopwords.words('english'))
    extended_stop_words = en_stop_words | \
                        {
                            '&amp;', 'rt',                           
                            'th','co', 're', 've', 'kim', 'daca'
                        }
    url_re = '(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'        
    df_copy['preprocessed_' + tweet_col] = df_copy['preprocessed_' + tweet_col].apply(lambda row: ' '.join([word for word in row.split() if (not word in stop_words) and (not re.match(url_re, word))]))
    
    # tokenize the tweets
    tokenizer = RegexpTokenizer('[a-zA-Z]\w+\'?\w*')
    df_copy['tokenized_' + tweet_col] = df_copy['preprocessed_' + tweet_col].apply(lambda row: tokenizer.tokenize(row))
    
    df_copy = preprocess2(df_copy)
    
    return df_copy

In [92]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


In [109]:
df = papers.head(10)
df_copy = df.copy()

# lower the tweets
df_copy['col2'] = df_copy['paper_text'].str.lower()

# replace \n with space
df_copy = df_copy.replace(to_replace='\n',value=' ',regex=True)

# expand contractions
df_copy['col2'] = df_copy['col2'].apply(lambda x: decontracted(x))

# filter out stop words and URLs
en_stop_words = set(stopwords.words('english'))
extended_stop_words = en_stop_words | \
                {
                        '&amp;', 'rt',                           
                        'th','co', 're', 've', 'kim', 'daca'
                }
url_re = '(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'        
df_copy['col2'] = df_copy['col2'].apply(lambda row: ' '.join([word for word in row.split() if (not word in extended_stop_words) and (not re.match(url_re, word))]))


df_copy['col2'] = df_copy['col2'].map(lambda x: re.sub(r'\W+', ' ', x))
df_copy['col2'] = df_copy['col2'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')

# tokenize the tweets
# tokenizer = RegexpTokenizer('[a-zA-Z]\w+\'?\w*')
# df_copy['col3'] = df_copy['col2'].apply(lambda row: tokenizer.tokenize(row))
# Use English stemmer.
# from nltk.stem.snowball import SnowballStemmer
# stemmer = SnowballStemmer("english")
# df_copy['col3'] = df_copy['col3'].apply(lambda x: [stemmer.stem(y) for y in x])

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

df['d3'] = df_copy['col2'].apply(lemmatize_text)

df_copy['col3'][1]

<ipython-input-109-bd1563f91cff>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['d3'] = df_copy['col2'].apply(lemmatize_text)


KeyError: 'col3'

[nltk_data] Downloading package wordnet to /home/giangvdq/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True